# p1-m2

bonita, rmt_052

This document is made to test 

# Model Inferences

### Let's load our modules first. 

In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings

from lol_features import LolPOVFeatures, get_patch_major, make_tokens


In [2]:
#this is for specific lgbm warning in inference. 
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names",
    category=UserWarning
)

### Then load model. 

In [3]:
pipe_cal_lgb_m = joblib.load("pipe_cal_lgb_m.joblib")
pipe_best_lr_m = joblib.load("pipe_best_lr_m.joblib")
pipe_best_lgb  = joblib.load("pipe_best_lgb.joblib")
pipe_best_lr   = joblib.load("pipe_best_lr.joblib")

## Our Database Setup
Assuming every new data added will go here. 

In [4]:
BONI_COLS = [
    "gameid","playoffs","league","date","game","patch","participantid","side","teamname","teamid",
    "firstPick","ban1","ban2","ban3","ban4","ban5",
    "pick1","pick2","pick3","pick4","pick5",
    "firstdragon",
    "goldat10","xpat10","csat10","killsat10","assistsat10","deathsat10",
    "opp_goldat10","opp_xpat10","opp_csat10","opp_killsat10","opp_assistsat10","opp_deathsat10",
    "result"
]

bonismatchlist_df = pd.DataFrame(columns=BONI_COLS)

In [5]:
def add_user_row(user_dict: dict, df: pd.DataFrame = None) -> pd.DataFrame:
    global bonismatchlist_df

    if df is None:
        df = bonismatchlist_df

    row = {c: user_dict.get(c, np.nan) for c in BONI_COLS}
    df.loc[len(df)] = row  # append

    #update bonismatchlist
    if df is bonismatchlist_df:
        bonismatchlist_df = df

    return df

### Our inferences data


In [6]:
#ps: results added for crosscheck, if unavailable, it will be listed unknown later
user_inputs = [
{
    "gameid": "LOLTMNT05_177060",
    "playoffs": 1,
    "league": "LEC",
    "date": "2026-02-22 18:41:27",
    "game": 1,
    "patch": "16.03",
    "participantid": 100,
    "side": "Blue",
    "teamname": "Team Vitality",
    "teamid": "oe:team:e8d8ae992fe72acb7adc7bfcee31dbb",
    "firstPick": 0.0,
    "ban1": "Varus", "ban2": "Azir", "ban3": "Orianna", "ban4": "Rumble", "ban5": "Nautilus",
    "pick1": "Ezreal", "pick2": "Ryze", "pick3": "Dr. Mundo", "pick4": "Ambessa", "pick5": "Karma",
    "firstdragon": 0.0,
    "goldat10": 17287.0, "xpat10": 21201.0, "csat10": 374.0,
    "killsat10": 2.0, "assistsat10": 2.0, "deathsat10": 1.0,
    "opp_goldat10": 16426.0, "opp_xpat10": 21063.0, "opp_csat10": 371.0,
    "opp_killsat10": 1.0, "opp_assistsat10": 1.0, "opp_deathsat10": 2.0,
    "result": 1
},
{
    "gameid": "LOLTMNT05_177060",
    "playoffs": 1,
    "league": "LEC",
    "date": "2026-02-22 18:41:27",
    "game": 1,
    "patch": "16.03",
    "participantid": 200,
    "side": "Red",
    "teamname": "Karmine Corp",
    "teamid": "oe:team:f3aa9bbd924ce3af42ac53f8326bc7c",
    "firstPick": 1.0,
    "ban1": "Pantheon", "ban2": "Gwen", "ban3": "Nami", "ban4": "Bard", "ban5": "Renekton",
    "pick1": "Yunara", "pick2": "Vi", "pick3": "Taliyah", "pick4": "Rek'Sai", "pick5": "Neeko",
    "firstdragon": 1.0,
    "goldat10": 16426.0, "xpat10": 21063.0, "csat10": 371.0,
    "killsat10": 1.0, "assistsat10": 1.0, "deathsat10": 2.0,
    "opp_goldat10": 17287.0, "opp_xpat10": 21201.0, "opp_csat10": 374.0,
    "opp_killsat10": 2.0, "opp_assistsat10": 2.0, "opp_deathsat10": 1.0,
    "result": 0
},
{
    "gameid": "LOLTMNT03_345495",
    "playoffs": 0,
    "league": "CBLOL",
    "date": "2026-02-22 18:53:07",
    "game": 4,
    "patch": "16.04",
    "participantid": 100,
    "side": "Blue",
    "teamname": "LØS",
    "teamid": "oe:team:ea65e6cc48a9be50fc5334369a5fca5",
    "firstPick": 1.0,
    "ban1": "Pantheon", "ban2": "Yunara", "ban3": "Renekton", "ban4": "Gwen", "ban5": "K'Sante",
    "pick1": "Aatrox", "pick2": "Ezreal", "pick3": "Alistar", "pick4": "Nocturne", "pick5": "Aurora",
    "firstdragon": 0.0,
    "goldat10": 16887.0, "xpat10": 21498.0, "csat10": 388.0,
    "killsat10": 1.0, "assistsat10": 2.0, "deathsat10": 1.0,
    "opp_goldat10": 16465.0, "opp_xpat10": 21261.0, "opp_csat10": 383.0,
    "opp_killsat10": 1.0, "opp_assistsat10": 0.0, "opp_deathsat10": 1.0,
    "result": 0
},
{
    "gameid": "LOLTMNT03_345495",
    "playoffs": 0,
    "league": "CBLOL",
    "date": "2026-02-22 18:53:07",
    "game": 4,
    "patch": "16.04",
    "participantid": 200,
    "side": "Red",
    "teamname": "FURIA",
    "teamid": "oe:team:87e3e1e75839f75a10bbc66e894d806",
    "firstPick": 0.0,
    "ban1": "Varus", "ban2": "Orianna", "ban3": "Ambessa", "ban4": "LeBlanc", "ban5": "Viktor",
    "pick1": "Leona", "pick2": "Syndra", "pick3": "Miss Fortune", "pick4": "Naafiri", "pick5": "Camille",
    "firstdragon": 1.0,
    "goldat10": 16465.0, "xpat10": 21261.0, "csat10": 383.0,
    "killsat10": 1.0, "assistsat10": 0.0, "deathsat10": 1.0,
    "opp_goldat10": 16887.0, "opp_xpat10": 21498.0, "opp_csat10": 388.0,
    "opp_killsat10": 1.0, "opp_assistsat10": 2.0, "opp_deathsat10": 1.0,
    "result": np.nan
},
{
    "gameid": "LOLTMNT05_178032",
    "playoffs": 1,
    "league": "LEC",
    "date": "2026-02-22 19:39:32",
    "game": 2,
    "patch": "16.03",
    "participantid": 100,
    "side": "Blue",
    "teamname": "Team Vitality",
    "teamid": "oe:team:e8d8ae992fe72acb7adc7bfcee31dbb",
    "firstPick": 0.0,
    "ban1": "Varus", "ban2": "Rumble", "ban3": "Sion", "ban4": "Ashe", "ban5": "Braum",
    "pick1": "Corki", "pick2": "Orianna", "pick3": "Yasuo", "pick4": "Nocturne", "pick5": "Rell",
    "firstdragon": 0.0,
    "goldat10": 16466.0, "xpat10": 21058.0, "csat10": 364.0,
    "killsat10": 1.0, "assistsat10": 0.0, "deathsat10": 3.0,
    "opp_goldat10": 17760.0, "opp_xpat10": 21162.0, "opp_csat10": 377.0,
    "opp_killsat10": 3.0, "opp_assistsat10": 3.0, "opp_deathsat10": 1.0,
    "result": 0
},
{
    "gameid": "LOLTMNT05_178032",
    "playoffs": 1,
    "league": "LEC",
    "date": "2026-02-22 19:39:32",
    "game": 2,
    "patch": "16.03",
    "participantid": 200,
    "side": "Red",
    "teamname": "Karmine Corp",
    "teamid": "oe:team:f3aa9bbd924ce3af42ac53f8326bc7c",
    "firstPick": 1.0,
    "ban1": "Pantheon", "ban2": "K'Sante", "ban3": "Gwen", "ban4": "Nami", "ban5": "Nautilus",
    "pick1": "Azir", "pick2": "Jarvan IV", "pick3": "Aurora", "pick4": "Xayah", "pick5": "Rakan",
    "firstdragon": 1.0,
    "goldat10": 17760.0, "xpat10": 21162.0, "csat10": 377.0,
    "killsat10": 3.0, "assistsat10": 3.0, "deathsat10": 1.0,
    "opp_goldat10": 16466.0, "opp_xpat10": 21058.0, "opp_csat10": 364.0,
    "opp_killsat10": 1.0, "opp_assistsat10": 0.0, "opp_deathsat10": 3.0,
    "result": 1
},
{
    "gameid": "LOLTMNT05_177061",
    "playoffs": 1,
    "league": "LEC",
    "date": "2026-02-22 20:35:48",
    "game": 3,
    "patch": "16.03",
    "participantid": 100,
    "side": "Blue",
    "teamname": "Team Vitality",
    "teamid": "oe:team:e8d8ae992fe72acb7adc7bfcee31dbb",
    "firstPick": 0.0,
    "ban1": "Varus", "ban2": "Aphelios", "ban3": "Bard", "ban4": "Sivir", "ban5": "Lucian",
    "pick1": "K'Sante", "pick2": "Seraphine", "pick3": "Xin Zhao", "pick4": "Twisted Fate", "pick5": "Braum",
    "firstdragon": 0.0,
    "goldat10": 17353.0, "xpat10": 20267.0, "csat10": 364.0,
    "killsat10": 2.0, "assistsat10": 2.0, "deathsat10": 3.0,
    "opp_goldat10": 18074.0, "opp_xpat10": 22132.0, "opp_csat10": 385.0,
    "opp_killsat10": 3.0, "opp_assistsat10": 5.0, "opp_deathsat10": 2.0,
    "result": 0
},
{
    "gameid": "LOLTMNT05_177061",
    "playoffs": 1,
    "league": "LEC",
    "date": "2026-02-22 20:35:48",
    "game": 3,
    "patch": "16.03",
    "participantid": 200,
    "side": "Red",
    "teamname": "Karmine Corp",
    "teamid": "oe:team:f3aa9bbd924ce3af42ac53f8326bc7c",
    "firstPick": 1.0,
    "ban1": "Kai'Sa", "ban2": "Pantheon", "ban3": "Jayce", "ban4": "Ashe", "ban5": "Nautilus",
    "pick1": "Rumble", "pick2": "Wukong", "pick3": "Ahri", "pick4": "Zeri", "pick5": "Alistar",
    "firstdragon": 1.0,
    "goldat10": 18074.0, "xpat10": 22132.0, "csat10": 385.0,
    "killsat10": 3.0, "assistsat10": 5.0, "deathsat10": 2.0,
    "opp_goldat10": 17353.0, "opp_xpat10": 20267.0, "opp_csat10": 364.0,
    "opp_killsat10": 2.0, "opp_assistsat10": 2.0, "opp_deathsat10": 3.0,
    "result": 1
}, 
{
    "gameid": "LOLTMNT05_171038",
    "playoffs": 0,
    "league": "LIT",
    "date": "2026-01-08 17:08:27",
    "game": 1,
    "patch": "16.01",
    "participantid": 100,
    "side": "Blue",
    "teamname": "GMBLERS Esports",
    "teamid": "oe:team:eb2f7366b23f654102ef78cfbd9ab56",
    "firstPick": 1.0,
    "ban1": "Zyra", "ban2": "Lillia", "ban3": "Wukong", "ban4": "Veigar", "ban5": "Viktor",
    "pick1": "Xin Zhao", "pick2": "Gnar", "pick3": "Corki", "pick4": "Ahri", "pick5": "Leona",
    "firstdragon": 1.0,
    "goldat10": 17079.0, "xpat10": 19437.0, "csat10": 309.0,
    "killsat10": 5.0, "assistsat10": 6.0, "deathsat10": 6.0,
    "opp_goldat10": 17617.0, "opp_xpat10": 20161.0, "opp_csat10": 346.0,
    "opp_killsat10": 6.0, "opp_assistsat10": 2.0, "opp_deathsat10": 5.0,
    "result": np.nan #we don't have data of this yet.
}
]

### Adding User Input to DF

In [7]:
for u in user_inputs:
    bonismatchlist_df = add_user_row(u)

print("Total rows in bonismatchlist_df:", bonismatchlist_df.shape[0])
bonismatchlist_df.tail(9)

Total rows in bonismatchlist_df: 9


,gameid,playoffs,league,date,game,patch,participantid,side,teamname,teamid,...,killsat10,assistsat10,deathsat10,opp_goldat10,opp_xpat10,opp_csat10,opp_killsat10,opp_assistsat10,opp_deathsat10,result
0,LOLTMNT05_177060,1,LEC,2026-02-22 18:41:27,1,16.03,100,Blue,Team Vitality,oe:team:e8d8ae992fe72acb7adc7bfcee31dbb,...,2.0,2.0,1.0,16426.0,21063.0,371.0,1.0,1.0,2.0,1.0
1,LOLTMNT05_177060,1,LEC,2026-02-22 18:41:27,1,16.03,200,Red,Karmine Corp,oe:team:f3aa9bbd924ce3af42ac53f8326bc7c,...,1.0,1.0,2.0,17287.0,21201.0,374.0,2.0,2.0,1.0,0.0
2,LOLTMNT03_345495,0,CBLOL,2026-02-22 18:53:07,4,16.04,100,Blue,LØS,oe:team:ea65e6cc48a9be50fc5334369a5fca5,...,1.0,2.0,1.0,16465.0,21261.0,383.0,1.0,0.0,1.0,0.0
3,LOLTMNT03_345495,0,CBLOL,2026-02-22 18:53:07,4,16.04,200,Red,FURIA,oe:team:87e3e1e75839f75a10bbc66e894d806,...,1.0,0.0,1.0,16887.0,21498.0,388.0,1.0,2.0,1.0,NaN
4,LOLTMNT05_178032,1,LEC,2026-02-22 19:39:32,2,16.03,100,Blue,Team Vitality,oe:team:e8d8ae992fe72acb7adc7bfcee31dbb,...,1.0,0.0,3.0,17760.0,21162.0,377.0,3.0,3.0,1.0,0.0
5,LOLTMNT05_178032,1,LEC,2026-02-22 19:39:32,2,16.03,200,Red,Karmine Corp,oe:team:f3aa9bbd924ce3af42ac53f8326bc7c,...,3.0,3.0,1.0,16466.0,21058.0,364.0,1.0,0.0,3.0,1.0
6,LOLTMNT05_177061,1,LEC,2026-02-22 20:35:48,3,16.03,100,Blue,Team Vitality,oe:team:e8d8ae992fe72acb7adc7bfcee31dbb,...,2.0,2.0,3.0,18074.0,22132.0,385.0,3.0,5.0,2.0,0.0
7,LOLTMNT05_177061,1,LEC,2026-02-22 20:35:48,3,16.03,200,Red,Karmine Corp,oe:team:f3aa9bbd924ce3af42ac53f8326bc7c,...,3.0,5.0,2.0,17353.0,20267.0,364.0,2.0,2.0,3.0,1.0
8,LOLTMNT05_171038,0,LIT,2026-01-08 17:08:27,1,16.01,100,Blue,GMBLERS Esports,oe:team:eb2f7366b23f654102ef78cfbd9ab56,...,5.0,6.0,6.0,17617.0,20161.0,346.0,6.0,2.0,5.0,NaN


### Prediction Function
Function below includes feature engineering setups and call on proba.

In [8]:
def run_inference(*args):
    global bonismatchlist_df
    #parse args
    if len(args) == 2:
        df_all = bonismatchlist_df
        gameid, teamname = args
    elif len(args) == 3:
        df_all, gameid, teamname = args
    else:
        raise TypeError("Use run_inference(gameid, teamname) or run_inference(df, gameid, teamname)")

    #lookup row
    df_row = df_all[(df_all["gameid"] == gameid) & (df_all["teamname"] == teamname)].copy()
    if df_row.empty:
        raise ValueError(f"No match found for gameid={gameid}, teamname={teamname}")

    #if multiple, take first (or you can change this logic)
    df_row = df_row.iloc[[0]].reset_index(drop=True)

    #engineered features
    df_row["golddiffat10"] = df_row["goldat10"].fillna(0) - df_row["opp_goldat10"].fillna(0)
    df_row["xpdiffat10"]   = df_row["xpat10"].fillna(0)  - df_row["opp_xpat10"].fillna(0)
    df_row["csdiffat10"]   = df_row["csat10"].fillna(0)  - df_row["opp_csat10"].fillna(0)

    df_row["killsdiffat10"]   = df_row["killsat10"].fillna(0)   - df_row["opp_killsat10"].fillna(0)
    df_row["assistsdiffat10"] = df_row["assistsat10"].fillna(0) - df_row["opp_assistsat10"].fillna(0)

    df_row["gold_per_kill_team"] = df_row["goldat10"].fillna(0) / (df_row["killsat10"].fillna(0) + 1)
    df_row["gold_per_kill_opp"]  = df_row["opp_goldat10"].fillna(0) / (df_row["opp_killsat10"].fillna(0) + 1)
    df_row["gpkdiffat10"]        = df_row["gold_per_kill_team"] - df_row["gold_per_kill_opp"]

    if "firstkill" not in df_row.columns:
        df_row["firstkill"] = 0
    df_row.loc[
        (df_row["killsat10"].fillna(0) == 0) & (df_row["opp_killsat10"].fillna(0) == 0),
        "firstkill"
    ] = 0

    if "firstdragon" not in df_row.columns:
        df_row["firstdragon"] = 0
    df_row["firstdragon"] = df_row["firstdragon"].fillna(0)

    #models columns
    intr_cols = [
        "gameid","participantid","date","game","teamname",
        "side","patch","playoffs","firstPick",
        "ban1","ban2","ban3","ban4","ban5",
        "pick1","pick2","pick3","pick4","pick5",
        "goldat10","xpat10","csat10","killsat10","assistsat10","deathsat10"
    ]

    match_cols = [
        "gameid","participantid","date","game","teamname",
        "side", "league", "patch","playoffs","firstPick",
        "ban1","ban2","ban3","ban4","ban5",
        "pick1","pick2","pick3","pick4","pick5",
        "firstdragon",
        "golddiffat10","xpdiffat10","csdiffat10",
        "killsdiffat10","assistsdiffat10","gpkdiffat10",
        "firstkill"
    ]

    X_intr = df_row[intr_cols].copy()
    X_match = df_row[match_cols].copy()

    #predict proba
    p_intr_lgb  = pipe_best_lgb.predict_proba(X_intr)[:, 1][0]
    p_intr_lr   = pipe_best_lr.predict_proba(X_intr)[:, 1][0]
    p_match_lgb = pipe_cal_lgb_m.predict_proba(X_match)[:, 1][0]
    p_match_lr  = pipe_best_lr_m.predict_proba(X_match)[:, 1][0]

    #actual results IF THERE IS
    actual_result = df_row.loc[0, "result"]
    if pd.isna(actual_result):
        result_display = "NYA"
    elif int(actual_result) == 1:
        result_display = "Win"
    else:
        result_display = "Lose"
    
    return {
        "gameid": gameid,
        "teamname": teamname,
        "winning i_lgbm": round(float(p_intr_lgb), 4),
        "winning i_logreg": round(float(p_intr_lr), 4),
        "winning m_lgbm": round(float(p_match_lgb), 4),
        "winning m_logreg": round(float(p_match_lr), 4),
        "result": result_display
    }

### Now, we predict!

In [9]:
##FOR ALL IN LIST. 
# results = []

# for u in user_inputs:
#     gameid = u["gameid"]
#     teamname = u["teamname"]
#     results.append(run_inference(gameid, teamname))

# results

Let's say we just want to know 1 specific team's win probability in a specific match (this is why we need to call it using gameid and teamname). GameID is never the same, only shared by 2 team facing each other. 

PS: the results indicates actual match results, it's used as crosscheck with our model. In cases where results is NOT YET AVAILABLE. It will return, NYA. 

**REMEMBER** As most sports prediction, we are using threshold=.5; it's usually mentioned as "team-Q probability to win is at n%." They do not classify it as win or lose, but quantified percentage. Hence the results below depict that. 

In [10]:
#NYA results
run_inference("LOLTMNT03_345495", "FURIA")

{'gameid': 'LOLTMNT03_345495',
 'teamname': 'FURIA',
 'winning i_lgbm': 0.7458,
 'winning i_logreg': 0.5682,
 'winning m_lgbm': 0.5797,
 'winning m_logreg': 0.4583,
 'result': 'NYA'}

In [11]:
run_inference("LOLTMNT05_178032", "Karmine Corp")

{'gameid': 'LOLTMNT05_178032',
 'teamname': 'Karmine Corp',
 'winning i_lgbm': 0.8763,
 'winning i_logreg': 0.7873,
 'winning m_lgbm': 0.7526,
 'winning m_logreg': 0.8009,
 'result': 'Win'}

# Results Summary and Explanation
How you read the results is basically, eg:
1. At 10-min marks, using FURIA's draft and early performance, they have around 58% probability to win. However, in comparison to their opponent's performance, their chances of winning is around 45%. [We don't know their actual results yet, hence NYA. This is also interesting because this drops basically saying: despite the early advantage, they still are predicted to lose. The proba results aren't very consistent here, especially on the matchup. As explained in the documentation, for matchup, when the results are contradicting each other, we should rely on logres more].

2. Following the drafting and their 10mins performance, Karmine Corp is expected to win around 87%. This holds true in comparison to their opponent at the same 10-mins mark, at 75-80% chance. [That prediction is accurate, in real life, their result was actually a win as well. The proba results are also coherent between the two team, as explained in the model documentation].